imports

In [ ]:
# imports 
import pandas as pd
import numpy as np
import json

loading data

In [ ]:
# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# extract lorry data from json
lorry_data = json.load(open('SaO_Optilandia_resub_depot_lorries.json', 'r'))

# set count to 0
k = 0

# initialise lorry list
lorry = []

# loop -> set i to the respective lorry key
for i in lorry_data.keys():
    # set j to the the number of lorries at key 'i'
    for j in range(0, len(lorry_data[i])):
        # append each lorry in lorry_data to lorry list
        lorry.append(pd.DataFrame(lorry_data[i][j], index=[k]))
        # accumulate index
        k += 1

# lorry dataframe
lorry_df = pd.concat(lorry)

In [ ]:
# setting required column 
location_df['required'] = location_df['capacity']-location_df['level']

# displaying rows where is_customer true
location_df[location_df['is_customer']==True].head()

In [ ]:
# add 'depot' column to lorry_df and updae with the lorry's relative depot 
lorry_df['depot'] = lorry_df.lorry_id.apply(lambda x: x.split('-')[0])
lorry_df.head()

In [ ]:
# list of depot locations (where nodes == depot)
depot_locations = np.where(location_df.is_depot)[0]

# list of customer locations (where nodes == customers)
customer_locations = np.where(location_df.is_customer)[0]

imports + visualising the map

In [ ]:
# imports
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt 

# parwise distance calculation for each node
euclidean = squareform(pdist(location_df[['x', 'y']]))

# edges list initialisation
edges = []

# loop through links_df rows
for _, (i, j) in links_df.iterrows():
    # append node at i, node at j, and their pairwise distance to edges
    edges.append((i, j, euclidean[i, j]))

# pos dict intialisation
pos = {}

# loop through location_df rows
for k, v in location_df[['x', 'y']].iterrows():
    # update pos dict with array of k, v 
    pos.update({k:v.values})

# initialise depot_labels dict
depot_labels = {}

# loop throgugh depot_locations
for i in depot_locations:
    # update depot_labels dict with {i:i}
    depot_labels.update({i:i})

# initialise customer_labels dict
customer_labels = {}

# loop through customer_locations
for i in customer_locations:
    # update customer_labels dict with {i:i}
    customer_labels.update({i:i})

# initialise nx Graph
G = nx.Graph()

# feed node list to G
G.add_nodes_from(location_df['id'].to_numpy())

# feed edges list to G
G.add_weighted_edges_from(edges)

# resize figure 
plt.figure(figsize=(16, 8))

# sketch graph
nx.draw(G, pos=pos, node_size=40)

# label depot nodes
nx.draw_networkx_labels(G, pos, depot_labels)

# label customer nodes
nx.draw_networkx_labels(G, pos, customer_labels)

# mark depot nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=depot_locations, node_color='r', node_size=400, alpha=0.9)

# mark customer nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=customer_locations, node_color='g', node_size=200, alpha=0.3)

clustering nodes ... (nearest-neighbour approach)

In [ ]:
# intialise cluster dict
cluster = {124:[], 127:[], 167:[], 523:[]}

# intialise nodes list
nodes = [] 

# loop through each node in customer_locations
for node in customer_locations:
    # check if node in nodes
    if node not in nodes:
        # initialise dist list
        dist = []
        # loop through each depot key
        for depot in cluster.keys():
            # append euclidean weights to dist 
            dist.append(euclidean[node, depot])
        # get shortest distance
        shortestDist = min(dist)
        # match shortest distance to equivalent node index
        nearestDepotIndex = np.where(euclidean[node]==shortestDist)
        # add node to relative nearest depot location
        cluster[int(nearestDepotIndex[0])].append(node)
        # track applied nodes
        nodes.append(node)
        # clear dist
        dist.clear()

# print allocated nodes to relative cluster points (depot locations)
print(cluster)

# clear nodes list
nodes.clear()

(breadth-first-search used for pathfinding)

In [ ]:
# Routing from A to B using recursive Breadth-First-Search based algorithm (pathfinding)
def routing(currState, toState, edges):
    # intialise visitedState list for tracking node traversal
    visitedState = [currState]
    # initialise visitedEdge list for tracking edge traversal
    visitedEdge = []
    # intialise edgeMemory list for storing the explored edges
    edgeMemory = []
    # intialise queue list for choosing the central node for next traversal
    queue = [currState]

    # loop while toState is not found
    while currState != toState:
        # remove and store the last element of the queue list as q
        q = queue.pop(0)
        
        # intialise currEdges dict which holds the next set of edges for traversals
        currEdges = {}
        # get the nodes at each edge, where either nodes are equivalent to q
        for edge in list(np.where(links_df[['id1', 'id2']]==q)[0]):
            # update the dict with the relative edge key and the node pairs
            currEdges.update({edge:[edges[edge][0], edges[edge][1]]})
        
        # store the explored edges in edgeMemory list
        edgeMemory.append(currEdges)

        # loop through each edge in currEdges
        for edge in currEdges:
            # check if the edge has been visited 
            if edge not in visitedEdge:
                # if not visited then add the edge to visitedEdge
                visitedEdge.append(edge)
                # check the node index in the edge that has not been visited 
                if currEdges[edge][0] not in visitedState and currEdges[edge][1] in visitedState:
                    # set currState to the unvisited node 
                    currState = currEdges[edge][0]
                    # mark the node in currState as visited 
                    visitedState.append(currState)
                    # add new currState to queue 
                    queue.append(currState)
                    # check if toState reached
                    if currState == toState:
                        # set currState to toState
                        currState = toState
                        # end loop
                        break 
                # similar to above but in the context of different index position of the node that has not been visited
                if currEdges[edge][1] not in visitedState and edges[edge][0] in visitedState:
                    currState = currEdges[edge][1]
                    visitedState.append(currState)
                    queue.append(currState)
                    if currState == toState:
                        currState = toState
                        break
    
    # set startState as the first node in visitedState list
    startState = visitedState[0]
    # set lastQ as the toState for tracking q node from end of order
    lastQ = [toState]
    # intialise backtrace list for backtracing the edges from edgeMemory
    backtrace = []
    # initialise nodetrace list for backtracing the nodes from edgeMemory
    nodetrace = []

    # intialise edgeMemoryReversed for reordering edgeMemory 
    edgeMemoryReversed = []
    # loop through each index between range 0 and length of edgeMemory
    for i in range (0, len(edgeMemory)):
        # set endElement to the last element in edgeMemory
        endElement = edgeMemory.pop(-1)
        # add the endElement to edgeMemoryReversed
        edgeMemoryReversed.append(endElement)
    
    # while last element in lastQ is not equivalent to the startState
    while lastQ[-1] != startState:
        # loop through each edge options in edgeMemoryReversed
        for edgeOpt in edgeMemoryReversed:
            # loop through each edge from as keys of the edge options
            for edge in edgeOpt.keys():

                # check if last element of lastQ is in the set of edge options given the edge
                if lastQ[-1] in edgeOpt[edge]:
                    # add the edge to backtrace 
                    backtrace.append(edge)
                    # check index of node which matches the lastQ element 
                    if lastQ[-1] == edgeOpt[edge][0] and lastQ[-1] != edgeOpt[edge][1]:
                        # update lastQ as the the node which does not match the lastQ element
                        lastQ.append(edgeOpt[edge][1])
                        # add the node to nodetrace
                        nodetrace.append(edgeOpt[edge][1])
                        # return to while iterate
                        break
                        # similar to above but in the context of different index postion of the matching node with lastQ element
                    if lastQ[-1] != edgeOpt[edge][0] and lastQ[-1] == edgeOpt[edge][1]:
                        lastQ.append(edgeOpt[edge][0])
                        nodetrace.append(edgeOpt[edge][0])
                        break
    
    # re-ordering edges from start to end
    edgeTraversed = []
    for i in range(0, len(backtrace)):
        endElement = backtrace.pop(-1)
        edgeTraversed.append(endElement)

    # re-ordering nodes from start to end 
    nodeOrder = []
    for i in range(0, len(nodetrace)):
        endElement = nodetrace.pop(-1)
        nodeOrder.append(endElement)
    
    # adding route weight (distance between nodes) to each traversal made
    routeWeight = []
    for edge in edgeTraversed:
        routeWeight.append(edges[edge][2])

    # return the the order in which nodes were visited and the order in which edges were traversed
    return nodeOrder, edgeTraversed, routeWeight

In [ ]:
# test: gives the node order in routing
print(f'node order: {routing(124, 10, edges)[0]}')
# test: gives the traversed edges in routing
print(f'edge order: {routing(124, 10, edges)[1]}')
# test: gives the weight routes for traversed
print(f'weights: {routing(124, 10, edges)[2]}')

genetic algorithm for exploring solution set

In [ ]:
# initailise routes dict 
routes = {}

# loop through lorry_df index
for i in lorry_df.index:
    # for each lorry, initialise journey by appending start (depot) and capacity (self)
    routes.update({lorry_df['lorry_id'][i]:[(lorry_df.depot[i]), lorry_df.capacity[i]]})

In [ ]:
# display routes so far
routes

In [ ]:
# imports 
import random

# function to generate a random path order (consider as genome) for each depot # returns a dict 
def randomPathArrangement(cluster):
    randomPathArr = {}
    customerAllocation = cluster
    for depot in depot_locations:
        randomPathArr.update({depot:random.sample(customerAllocation[depot], len(customerAllocation[depot]))})
    return randomPathArr

In [ ]:
print(randomPathArrangement(cluster))

In [ ]:
# function to generate multiple initial solutions (population) # returns genomes (individual solution) in a list
def population_init(cluster, size):
    population = []
    for i in range(0, size):
        population.append(randomPathArrangement(cluster))
    return population

In [ ]:
# for test
tempPop = population_init(cluster, 2)

In [ ]:
# for test
for i in tempPop:
    print(f'\ntempPop:\n{i}') 

In [ ]:
# return populationFitness at each index in ascending order (most fit to least fit)
def fitness(population, edges):

    genomeWeight = []
    popFitness = []

    # for a possible solution in the set of solutions (genome in population)
    for genome in population:
        # initialise genomeFitness dict to store distances at each depot per genome 
        genomeFitness = {}
        for depot in genome.keys():
            genomeFitness.update({depot:[]})
            nodes = genome[depot]
            for idx in range(len(nodes)-1):
                nodeA = nodes[idx]
                nodeB = nodes[idx+1]
                # get sum of each traversal distance occurence between two nodes | routing() enables this 
                distance = sum(routing(nodeA, nodeB, edges)[2])
                genomeFitness[depot].append(distance)
        # update popFitness list with each genome and relative fitness values
        genomeWeight.append(genomeFitness)

    for i in genomeWeight.keys():
        weight = genomeWeight[i]
        

            # maybe while loop here? i.e., while nodes fitness not finished eval... pathfind / get weight etc...

fitness(tempPop, edges)

In [ ]:
genomeWeight = [{124: [764.207424313277, 29.495691125645767, 301.94789440648617, 86.43188547460109, 103.29269324774098, 232.9515137867115, 128.22947032920757, 393.38798000038685, 709.8293768076302, 642.1342308631976, 200.88800501175132, 658.398565959393, 608.9983256352541, 300.02473294735586, 423.23102621587157, 297.6257062845869, 322.8917823783063, 288.27462421455766, 295.6832252823565, 450.6829775380554, 195.65720050348318, 678.7852237779401, 515.7349553313808, 335.91749025969284, 106.43963252156183, 307.76140737295805, 402.066079913012, 643.7135396049698, 658.8047018142415, 768.2248081159325, 813.708674571676, 103.92345376028538, 270.94705340955727, 589.2408968213057, 919.1345237215405, 654.2794983026115, 305.56720328099027, 257.9474081038173, 363.5496130322805, 552.4780590706876, 373.37014964551736], 127: [348.4142861903571, 357.05571442232014, 455.1483505812167, 704.0157008868272, 427.0068674109813, 123.42833303093978, 411.7488382645468, 431.657181083932, 574.4160028703847, 58.06471349776065, 310.7541074832516, 436.98530596353703, 627.5465252964724, 315.9747664603495, 290.38448478394196, 83.2969859130678, 301.470886946703, 457.1135264180963, 126.18430600481119, 314.8792897289423, 497.5738890194847], 167: [1250.272738188178, 140.32987717185813, 196.62339978472454, 279.53262240814354, 543.5169570778512, 444.5473336872827, 341.07549555855906, 731.1340300246841, 627.9768728760403, 413.3193194547188, 513.2440711414912, 765.334939304412, 103.26122839541503, 154.71975635332103, 866.2232691802731, 745.5637553522878, 436.52936832788794, 16.504197856127956, 409.1000414868827, 506.5964086798391, 560.3666894200053, 551.7322751837983, 455.2337440489541, 528.4273490154951, 390.7217940343114, 296.0146898312134, 352.2068989779755, 216.21920758157987, 380.7234794752683, 153.8802697173813, 423.6339377707202, 523.5703057251819, 900.3987567184421, 486.5244553942727, 496.7391510005817, 379.1737183516223, 856.9137593535364, 1039.3115132434327, 1172.69760327064], 523: [397.2511759610576, 76.22882539841802, 466.2029778486828, 436.8572098956855, 79.83964888478602, 49.42764494439111, 274.93391794914004, 440.3321915508013, 412.6407636008004, 515.8336379243157, 486.5016359164346, 437.12467956689915, 482.47541209418864, 643.3140561058268, 445.47109668411093, 719.9332877165486]}, {124: [750.643259987773, 423.37743893418207, 555.4173822966279, 156.98076369543847, 760.771364715029, 725.5565364882851, 857.3776595203979, 486.027363715945, 67.1646045395968, 251.40260165645648, 741.3529645615679, 894.0121350847274, 310.0942213005814, 323.99389891712786, 445.7457690739033, 312.6148483903993, 735.9124686435192, 784.8974894199679, 572.8960429850806, 303.68319466476635, 615.828068389548, 385.11928016382257, 404.06658076196885, 511.95764285989117, 328.2792896865652, 488.21507017087686, 197.85977686606589, 503.30633319757663, 579.5283660766967, 837.9551775675354, 436.5013682069731, 425.1640065860985, 522.498811954075, 787.5145000996667, 312.90159716303197, 240.33902220809955, 729.2575192358966, 1038.0932740504438, 330.19659478828333, 285.60808306511996, 153.6126644643762], 127: [677.8034563399877, 414.63152438896566, 505.5620030294063, 721.5488497821568, 625.5988825280571, 201.52462678216648, 872.9933636781392, 567.1874348254692, 513.1793283617236, 316.9482949400491, 228.73157046141262, 211.53197688206873, 227.65493672471757, 386.5781652473159, 375.0142202335159, 221.92021248173802, 514.7583461373138, 762.63463460227, 775.4245703306068, 290.89450426696715, 309.8401754458338], 167: [653.7936625094658, 647.0908069935614, 388.9161571326128, 757.5469325178472, 657.5489231022322, 440.6037374319461, 775.2152616616496, 1210.5048782399704, 309.74619336118405, 307.15465183311096, 102.89481566627546, 481.29683062971213, 615.5378621639363, 867.4912934000644, 338.02376959713047, 326.8319432728658, 467.7193826820153, 671.3485984642659, 345.0405396199166, 509.26999488616707, 586.8072993867152, 518.1556336129592, 194.12618232326727, 900.3987567184421, 834.9360555774145, 438.2639443877282, 683.6775912149867, 546.6704085031886, 261.2587028961935, 344.0218101355311, 613.3210263784691, 308.9304150943407, 549.7801516148212, 405.5140922766085, 449.16223402335856, 466.9280602777058, 496.7391510005817, 1269.4427630456041, 369.79888183408667], 523: [596.9186469151281, 246.4890503947042, 389.6917744146752, 30.598838284648746, 233.05953860822746, 721.3618573224205, 631.1423831366382, 375.8297831879157, 109.13710916309944, 569.1121440495224, 643.3140561058268, 243.54765886592168, 206.73743581900288, 344.5232130924697, 620.3572060416097, 515.8336379243157]}]
popWeight = []
for genome in genomeWeight:
    print(f'\n')
    for depot in genome.keys():
        #print(depot)
        popWeight.append({depot:sum(genome[depot])})

print(popWeight)